In [ ]:
import kratos
from concurrent.futures import ThreadPoolExecutor, wait
from agno.agent import Agent
from agno.models.lmstudio import LMStudio
from agno.tools.youtube import YouTubeTools
from agno.tools.yfinance import YFinanceTools
from agno.tools.duckduckgo import DuckDuckGoTools
from functools import partial

# Creating a few agents using the Agno framework for building agents
# All the llm models here use LMStudio as the provider and connect to local API server

web_agent = Agent(
    model=LMStudio(
        id="deepseek/deepseek-r1-0528-qwen3-8b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosWebSearch",
    tools=[DuckDuckGoTools()],
    instructions="You are Kratos Web Search Agent, specialized in finding current information from the web. Use DuckDuckGo to search for the latest news, facts, and information. Always provide up-to-date and accurate information from reliable sources. Be concise and focus on delivering the most relevant search results.",
    show_tool_calls=False,
)
web_agent_partial = partial(kratos.submit, web_agent, "KratosWebSearch", dependencies=["ddgs", "duckduckgo-search"])


finance_agent = Agent(
    model=LMStudio(
        id="mlx-community/Llama-3.2-3B-Instruct-4bit",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosFinance",
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="You are Kratos Finance Agent, specialized in stock market analysis and financial data. Use YFinance to get current stock prices, analyst recommendations, and company information. When users ask about stocks, always provide current prices and relevant financial metrics. Be precise with numbers and explain what the data means for investment decisions.",
    show_tool_calls=False,
)
finance_agent_partial = partial(kratos.submit, finance_agent, "KratosFinance", dependencies=["yfinance"])


youtube_agent = Agent(
    model=LMStudio(
        id="qwen/qwen3-4b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosYouTube",
    tools=[YouTubeTools()],
    instructions="You are Kratos YouTube Agent, specialized in analyzing YouTube videos and transcripts. Use YouTube tools to extract video information, transcripts, and analyze content. When users mention YouTube videos, provide detailed analysis of the content, key points, and insights from the transcript. Be thorough in your analysis and highlight important information.",
    show_tool_calls=False,
)
youtube_agent_partial = partial(kratos.submit, youtube_agent, "KratosYouTube", dependencies=["youtube-transcript-api"])

In [ ]:
# Submit these agents so that they are ready to be used at a later point - none of
# these models are actually loaded in memory right now and thus no resources are wasted.
with ThreadPoolExecutor(max_workers=3) as executor:
    print("Submitting agents...")
    futures = [
        executor.submit(agent_partial)
        for agent_partial in [web_agent_partial, finance_agent_partial, youtube_agent_partial]
    ]

    wait(futures)
    print("Agents submitted and are ready to be invoked!")